嵌入（embedding）在表征学习中是一个非常重要的一个概念。什么是嵌入呢？简而言之，嵌入就是一个低维稠密的向量。在图机器学习中，现在一个流行的范式是，将图中的每个节点映射为一个嵌入，然后将节点的嵌入输入到下游的机器学习任务中。图中每个节点的嵌入要反映出图的结构信息与节点的属性信息。怎么样得到节点的嵌入呢？我们可以通过随机游走或者深度学习的方法。由于以上两种方法得到嵌入的方法有些复杂，我们的task_1将从一个最简单的图嵌入学习方法入手，带你体会学习图节点嵌入学习的美妙！

在task_1中,我们将为学习节点嵌入编写一个完整的pipeline。

该图嵌入学习方法的总体思路是：先为图中的每一个节点分配一个随机嵌入，然后不断优化更新这个嵌入。

优化具体实现过程是：我们将图中真实存在的边称作正边，设置标签为1，然后随机采样图中与正边数量相同的负边，设置标签为0，然后进行传统的监督学习任务。

该task分为3个步骤：

首先，我们将加载一个的经典图，[Karate Club Network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club)。我们将探索该图的多个图统计信息。

然后，再将图结构转换为PyTorch张量，这样我们就可以对图进行机器学习。

最后，我们将完成第一个关于图节点嵌入的学习算法

现在让我们开始吧！

## Zachary's karate club Graph   
我们先通过networkX包载入一个图[Karate Club Network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club)，该图描述了空手道俱乐部34名成员的社交网络

In [1]:
import networkx as nx
G = nx.karate_club_graph()

print(G)
print(type(G))
print(G.edges)
print(G.nodes)

Graph named "Zachary's Karate Club" with 34 nodes and 78 edges
<class 'networkx.classes.graph.Graph'>
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 10), (0, 11), (0, 12), (0, 13), (0, 17), (0, 19), (0, 21), (0, 31), (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 30), (2, 3), (2, 7), (2, 8), (2, 9), (2, 13), (2, 27), (2, 28), (2, 32), (3, 7), (3, 12), (3, 13), (4, 6), (4, 10), (5, 6), (5, 10), (5, 16), (6, 16), (8, 30), (8, 32), (8, 33), (9, 33), (13, 33), (14, 32), (14, 33), (15, 32), (15, 33), (18, 32), (18, 33), (19, 33), (20, 32), (20, 33), (22, 32), (22, 33), (23, 25), (23, 27), (23, 29), (23, 32), (23, 33), (24, 25), (24, 27), (24, 31), (25, 31), (26, 29), (26, 33), (27, 33), (28, 31), (28, 33), (29, 32), (29, 33), (30, 32), (30, 33), (31, 32), (31, 33), (32, 33)]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


## Graph to Tensor

In [2]:
import torch
def graph_to_edge_list(G):
  # 功能: 该函数返回图的edge list，输入为nxworkX
  # 中的图。返回的edge_list是由元组组成的列表，其
  # 中每个元组由边的两个节点组成

  edge_list = []

  ############# 在这儿写下你的代码吧 ############
  for edge in G.edges():
    edge_list.append(edge)
  #########################################

  return edge_list

def edge_list_to_tensor(edge_list):
  # 功能: 将edge list转换为张量，该张量的shape为[2 x len(edge_list)]

  edge_index = torch.tensor([])

  ############# 在这儿写下你的代码吧 ############
  edge_index = torch.tensor(edge_list)
  #########################################

  return edge_index.t()

import networkx as nx
G = nx.karate_club_graph()
print(edge_list_to_tensor(graph_to_edge_list(G)))



tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  4,
          4,  5,  5,  5,  6,  8,  8,  8,  9, 13, 14, 14, 15, 15, 18, 18, 19, 20,
         20, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 26, 26, 27, 28, 28, 29,
         29, 30, 30, 31, 31, 32],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  2,  3,
          7, 13, 17, 19, 21, 30,  3,  7,  8,  9, 13, 27, 28, 32,  7, 12, 13,  6,
         10,  6, 10, 16, 16, 30, 32, 33, 33, 33, 32, 33, 32, 33, 32, 33, 33, 32,
         33, 32, 33, 25, 27, 29, 32, 33, 25, 27, 31, 31, 29, 33, 33, 31, 33, 32,
         33, 32, 33, 32, 33, 33]])


In [3]:
# note：负边就是图中两个未连接的节点形成的边。正边是图中真实存在的边，
# 正边由一个元组表示：（节点1，节点2），节点1与节点2相连；而负边是图
# 中不存在的边，负边也由一个元组表示：（节点1，节点2），而节点1与节点
# 2在图中不相连

import random

def sample_negative_edges(G, num_neg_samples):
  # 功能: 随机采样num_neg_samples个负边。该函数
  # 返回一个由负边（negative edge）构成的list
  
  neg_edge_list = []
  ############# 在这儿写下你的代码吧 ############
  node = list(G.nodes)
  node_pairs = []
  for i in range(len(node)):
      for j in range(i+1, len(node)):
          node_pairs.append((node[i], node[j]))

    
    # 从节点对中随机选择负边
  while len(neg_edge_list) != num_neg_samples:  
      pair = random.choice(node_pairs)
      if not G.has_edge(*pair):  # 检查节点是否相连
          neg_edge_list.append(pair)


  #########################################
  
  return neg_edge_list



现在，是时候为我们的图创建节点嵌入矩阵了！我们希望空手道俱乐部网络中的每个节点嵌入的维度都是16维。并且我们规定初始嵌入中的每个数值都服从均匀分布。

In [4]:
import torch.nn as nn
torch.manual_seed(1)

def create_node_emb(num_node=34, embedding_dim=16):
  # 功能: 创建一个节点嵌入矩阵，并随机化
  emb = nn.Embedding(num_node, embedding_dim)
  emb.weight.data = torch.rand(num_node, embedding_dim)
  return emb

emb = create_node_emb()

# 打印嵌入层
print("Embedding: {}".format(emb))

Embedding: Embedding(34, 16)


In [5]:
# 训练过程
from torch.optim import SGD
import torch.nn as nn

def accuracy(pred, label):
  # 功能: 利用pred tensor (after sigmoid)和label tensor 
  # (torch.LongTensor)去计算预测准确率。如果预测的值大于
  # 0.5被视为分类1，否则被视为分类0。准确率保留4位小数

  accu = 0.0

  ############# 在这儿写下你的代码吧 ############
  pred_label = (pred > 0.5).long()
  accu = (pred_label == label).sum().item()/len(label)
  accu = round(accu , 4)

  #########################################

  return accu

def train(emb, loss_fn, sigmoid, train_label, train_edge):
  # 功能: 训练节点嵌入层 
  # 提示：
  # (1) 得到train_edge的nodes的嵌入
  # (2) 将train_edge中的节点对的嵌入做点积
  # (3) 将点积输入sigmoid函数得到sigmoid output
  # (4) 将sigmoid output和标签输入输入loss_fn中算损失
  # (5) 每轮（epoch）训练将打印ccuracy和loss
  # (6) 使用损失函数与优化器更新每个节点的embedding

  epochs = 500
  learning_rate = 0.1

  optimizer = SGD(emb.parameters(), lr=learning_rate, momentum=0.9)

  for i in range(epochs):
    ############# 在这儿写下你的代码吧 ############
    optimizer.zero_grad()
    #(1)
    embednode = emb(train_edge)
    #(2)
    dot = embednode[0].mul(embednode[1])
    dot = torch.sum(dot,1)
    #(3)
    sigmoid_output = sigmoid(dot)
    #(4)
    loss = loss_fn(sigmoid_output, train_label)
    #(5)
    accu = accuracy(sigmoid_output,train_label)
    print(f"Epoch {i+1}, Accuracy: {accu}, Loss: {loss.item()}")
    #(6)
    loss.backward()
    optimizer.step()

    #########################################
  
  

In [6]:
# 开始引用上面我们构造的函数进行训练啦,这相当于main()函数


# pos_edge_list为正边(positive edge)的list
pos_edge_list = graph_to_edge_list(G)
# 将pos_edge_list转换成张量得到pos_edge_index
pos_edge_index = edge_list_to_tensor(pos_edge_list)

# 随机采样78条负边，neg_edge_list为负边的list
neg_edge_list = sample_negative_edges(G, len(pos_edge_list))
neg_edge_index = edge_list_to_tensor(neg_edge_list)

# 产生正负标签
pos_label = torch.ones(pos_edge_index.shape[1])
neg_label = torch.zeros(neg_edge_index.shape[1] )

# 将正负标签拼接成一个张量
train_label = torch.cat([pos_label, neg_label], dim=0)

# 将正负边拼接成一个张量
train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

# 训练更新节点嵌入
loss_fn = nn.BCELoss()
sigmoid = nn.Sigmoid()
train(emb, loss_fn, sigmoid, train_label, train_edge)






Epoch 1, Accuracy: 0.5, Loss: 2.0075459480285645
Epoch 2, Accuracy: 0.5, Loss: 1.9931122064590454
Epoch 3, Accuracy: 0.5, Loss: 1.9659230709075928
Epoch 4, Accuracy: 0.5, Loss: 1.9276773929595947
Epoch 5, Accuracy: 0.5, Loss: 1.880060076713562
Epoch 6, Accuracy: 0.5, Loss: 1.8247097730636597
Epoch 7, Accuracy: 0.5, Loss: 1.763195276260376
Epoch 8, Accuracy: 0.5, Loss: 1.6970016956329346
Epoch 9, Accuracy: 0.5, Loss: 1.6275155544281006
Epoch 10, Accuracy: 0.5, Loss: 1.5560158491134644
Epoch 11, Accuracy: 0.5, Loss: 1.483666181564331
Epoch 12, Accuracy: 0.5, Loss: 1.411506175994873
Epoch 13, Accuracy: 0.5, Loss: 1.3404464721679688
Epoch 14, Accuracy: 0.5, Loss: 1.2712653875350952
Epoch 15, Accuracy: 0.5, Loss: 1.2046048641204834
Epoch 16, Accuracy: 0.5, Loss: 1.1409746408462524
Epoch 17, Accuracy: 0.5, Loss: 1.0807552337646484
Epoch 18, Accuracy: 0.5064, Loss: 1.0242061614990234
Epoch 19, Accuracy: 0.5192, Loss: 0.9714784026145935
Epoch 20, Accuracy: 0.5192, Loss: 0.9226255416870117
Epoc